# 🚀 Focoos HUB Integration

This notebook demonstrates how to use FocoosHUB to interact with the Focoos AI platform.
FocoosHUB provides a seamless integration between your local environment and Focoos cloud services,
allowing you to:
- Access and manage your user account and API credentials
- List, download and deploy remote models from the Focoos model registry
- Upload and manage your custom trained models
- Run cloud inference on managed models
- Monitor model performance and usage metrics

## 🐍 Setup Focoos

In [ ]:
%pip install 'focoos @ git+https://github.com/FocoosAI/focoos.git'

In [ ]:
import os

API_KEY = os.getenv(
    "FOCOOS_API_KEY"
)  # write here your API key os set env variable FOCOOS_API_KEY, will be load from env if not provided

## User info

In [ ]:
from focoos import FocoosHUB

hub = FocoosHUB(api_key=API_KEY)
user_info = hub.get_user_info()
user_info

## Remote Models

In [ ]:
from focoos import FocoosHUB

hub = FocoosHUB()
models = hub.list_remote_models()
models

## Remote Datasets

###

In [ ]:
from focoos import FocoosHUB

hub = FocoosHUB()
datasets = hub.list_remote_datasets(include_shared=True)
datasets

## Get Model Info

In [ ]:
from focoos import FocoosHUB

model_ref = None  # place here the ref of the model you want to retrieve

hub = FocoosHUB()
if model_ref is not None:
    model_info = hub.get_model_info(model_ref)

## Remote Inference with managed models

In [ ]:
from io import BytesIO

import requests
from PIL import Image

from focoos import FocoosHUB

model_ref = None  # place here the ref of the model you want to retrieve

response = requests.get("https://public.focoos.ai/samples/pexels-abby-chung.jpg")
image = Image.open(BytesIO(response.content))

hub = FocoosHUB()
if model_ref is not None:
    model = hub.get_remote_model(model_ref)
    results = model.infer(image=image, threshold=0.5, annotate=True)

    Image.fromarray(results.image)

# Local Training with **remote dataset** from HUB and push Model

In [ ]:
from focoos import DatasetSplitType, FocoosHUB, ModelManager, TrainerArgs
from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task

remote_dataset_ref = None  # place here the ref of the dataset you want to download

hub = FocoosHUB()
if remote_dataset_ref is not None:
    my_datasets = hub.list_remote_datasets(include_shared=False)
    remote_dataset = hub.get_remote_dataset(remote_dataset_ref)
    dataset_path = remote_dataset.download_data()
    auto_dataset = AutoDataset(dataset_name=dataset_path, task=remote_dataset.task, layout=remote_dataset.layout)

    train_augs, val_augs = get_default_by_task(remote_dataset.task, 640, advanced=False)
    train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
    valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

    model = ModelManager.get("fai-detr-l-obj365")

    args = TrainerArgs(
        run_name=f"{model.name}_{train_dataset.name}",
        output_dir="./experiments",
        amp_enabled=True,
        batch_size=16,
        max_iters=500,
        eval_period=50,
        learning_rate=0.0008,
        scheduler="MULTISTEP",
        weight_decay=0.02,
        workers=16,
        patience=1,
        sync_to_hub=True,  # use this to sync model info, weights and metrics on the hub
    )

    model.train(
        args, train_dataset, valid_dataset, hub=hub
    )  # Hub is optional, if not provided and sync_to_hub is True, will be created automatically